In [1]:
import pandas as pd
import numpy as np

# CUPED
**CUPED (Controlled-experiment Using Pre-Experiment Data or regression adjustment)** был разработан для уменьшения дисперсии путем включения данных, полученных до эксперимента, с помощью линейной регрессии. В отдельных случаях используются данные, которые собраны во время эксперименты, но они должны быть независимы от эксперимента. Один из способов проверки независимости - проверка ковариат одной группы с ковариатами другой через статистический тест.


$$Y_{cuped} = Y -\theta X$$
$\theta X$= линейная регрессия для одной ковариаты.
$\theta = \frac{cov(Y,X)}{var(x)}$ <br>
Требования:

- $E[X | W = 1] = E[X | W = 0]$ где W номер группы в тесте.


In [ ]:
def cuped(
        test_col: pd.Series,
        cov_col: pd.Series,
)->pd.Series:
        """ 
        Создает cuped-колонку 

        test_col (pd.Series): колонка с данными теста в обоих группах.
        cov_col (pd.Series) : колонка с данными ковариаты.

        return - готовая cuped-колонка.
        """

        covariance = np.cov(cov_col, test_col)[0, 1]
        variance = cov_col.var()
        theta = covariance / variance
        cuped_col = test_col - theta * cov_col

        return cuped_col


# CUPAC 
**CUPAC (Control Using Predictions as Covariates)** использует модели машинного обучения для предсказания Y, и использует Y_predicted в качестве ковариаты.
$$Y_{cupac} =Y-f(X_i) $$

Требования:
<br> для  $X| W = 1$ and $X | W = 0$, которые взяты во время эксперимента, должны иметь одинаковое распределения. Проверяется тестами на одинаковость распределений.<br>

Здесь будет предоставлена функция сразу с улучшенным CUPAC, который использует любую модель МО для предсказания Y, а после использует линейную регрессию для предсказания $Y_{cupac}$:
$$Y_{improvedCupac} =Y -\hat{f}(X_i)-\hat{\gamma} Z_i $$
$\hat{f}(X_i)$ - предсказанная Y с помощью доэкспериментальных ковариат. <br>
$\hat{\gamma}$ - предсказанная $\hat{R_i} = Y_i − \hat{f}(X_i)$.<br>
$Z_i$ - внутриэкспериментальные ковариаты. <br>
Подробности в статьях: https://arxiv.org/pdf/2410.09027 https://habr.com/ru/articles/869998/


In [ ]:
def improved_cupac(
    test_col: pd.Series,    
	pre_test_covs: pd.DataFrame,
	in_test_covs: pd.DataFrame,
	cupac_model
) -> pd.Series:
	"""
    Считает улучшенный CUPAC

    test_col (pd.Series): колонка с данными теста в обоих группах.
	pre_test_covs (pd.DataFrame): данные с доэкспериментальными ковариатами.
	in_test_covs: (pd.DataFrame): данные с внутриэкспериментальными ковариатами.
	cupac_model: объект модели МО.

	return - готовая CUPAC колонка.

    """

    #CUPAC
	model = cupac_model.fit(pre_test_covs, test_col) 
	Y_pred = model.predict(pre_test_covs)

	r = test_col - Y_pred

	#Improved CUPAC
	from sklearn.linear_model import LinearRegression
	lin_model = LinearRegression()	
	lin_model.fit(in_test_covs, r)
	gamma_hat = lin_model.coef_

	adj_R = r - in_test_covs.dot(gamma_hat)

	return adj_R